# Group 7 Tiny time mixer code. This should work on correctly set up data but I do not have our data so I can not vouch

### TODO Adapt real code from https://github.com/ibm-granite/granite-tsfm/blob/main/notebooks/tutorial/ttm_channel_mix_finetuning.ipynb

### TODO Get real data into Tidy CSV format (ie each row is a column)

### Current code is from https://colab.research.google.com/github/ibm-granite/granite-tsfm/blob/main/notebooks/hfdemo/ttm_getting_started.ipynb

In [1]:
#! conda update -n base -c defaults conda -y
#! conda install lxml -y

In [2]:
#! conda install pytorch pytorch-cuda=12.1 -c pytorch -c nvidia -y
#! pip install "tsfm_public[notebooks] @ git+https://github.com/ibm-granite/granite-tsfm.git@v0.2.14" -y

In [3]:
import math
import os
import tempfile
import pandas as pd

from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from transformers import EarlyStoppingCallback, Trainer, TrainingArguments, set_seed
from transformers.integrations import INTEGRATION_TO_CALLBACK

from tsfm_public import TrackingCallback, count_parameters, load_dataset
from tsfm_public.toolkit.get_model import get_model
from tsfm_public.toolkit.lr_finder import optimal_lr_finder
from tsfm_public.toolkit.visualization import plot_predictions

from tsfm_public import (
    TimeSeriesPreprocessor,
    TrackingCallback,
    count_parameters,
    get_datasets,
)

In [ ]:
path = "/scratch.global/csci8523_group_7/sst_data/2023_365_glsea_sst.nc"


In [4]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [14]:
import pandas as pd
#1995
url = "https://apps.glerl.noaa.gov/erddap/griddap/GLSEA_GCS.csvp?sst%5B(2023-12-31T12:00:00Z):1:(2023-12-31T12:00:00Z)%5D%5B(38.8749871947297):1:(50.6059751976539)%5D%5B(-92.4199507342304):1:(-75.8816402880531)%5D"#"https://apps.glerl.noaa.gov/erddap/griddap/GLSEA_GCS.csvp?sst%5B(1995-01-01T12:00:00Z):1:(2023-12-31T12:00:00Z)%5D%5B(42.05651963):1:(42.05649963)%5D%5B(-87.66870036):1:(-87.66867036)%5D"
data = pd.read_csv(url, on_bad_lines = "warn", delimiter=",", verbose = True)

Tokenization took: 34.89 ms
Type conversion took: 14.40 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 24.99 ms
Type conversion took: 15.44 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 25.06 ms
Type conversion took: 15.58 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 25.55 ms
Type conversion took: 15.59 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 24.25 ms
Type conversion took: 14.32 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 24.33 ms
Type conversion took: 15.75 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 24.86 ms
Type conversion took: 14.59 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 14.16 ms
Type conversion took: 7.99 ms
Parser memory cleanup took: 0.00 ms


In [ ]:
SST_FN = "GLSEA_SST_data.csv"
filtered_data = data.dropna()
filtered_data['time (UTC)'] = pd.to_datetime(filtered_data['time (UTC)'])
filtered_data = filtered_data.rename(columns={'time (UTC)': 'date', "sst (degree_C)": "SST"})
filtered_data = filtered_data.drop(["latitude (degrees_north)", "longitude (degrees_east)"], axis=1)
filtered_data.to_csv(SST_FN, index=False)#date
print(data.shape)#139598#989678
print(filtered_data.head())

(989678, 4)
                            date   SST
213288 2023-12-31 12:00:00+00:00  4.72
213289 2023-12-31 12:00:00+00:00  4.72
213290 2023-12-31 12:00:00+00:00  4.69
213291 2023-12-31 12:00:00+00:00  4.69
214467 2023-12-31 12:00:00+00:00  4.44


In [7]:
#Set seed for reproducibility
SEED = 42
set_seed(SEED)

# DATA ROOT PATH
# Make sure to download the target data (here ettm2) on the `DATA_ROOT_PATH` folder.
# ETT is available at: https://github.com/zhouhaoyi/ETDataset/tree/main
target_dataset = "SST"
DATA_ROOT_PATH = SST_FN

# Results dir
OUT_DIR = "ttm_finetuned_models/"


# Forecasting parameters
context_length = 512
forecast_length = 96

In [8]:
# Load the data file and see the columns
timestamp_column = "date"
# timestamp_column = "timestamp"
id_columns = []

data = pd.read_csv(
    DATA_ROOT_PATH,
    parse_dates=[timestamp_column],
)


data[timestamp_column] = pd.to_datetime(data[timestamp_column])
print(data.head())
column_specifiers = {
    "timestamp_column": timestamp_column,
    "id_columns": id_columns,
    "target_columns": ["SST"],
}

split_params = {"train": [0, 0.5], "valid": [0.5, 0.75], "test": [0.75, 1.0]}

                       date   SST
0 2023-12-31 12:00:00+00:00  4.72
1 2023-12-31 12:00:00+00:00  4.72
2 2023-12-31 12:00:00+00:00  4.69
3 2023-12-31 12:00:00+00:00  4.69
4 2023-12-31 12:00:00+00:00  4.44


In [9]:
tsp = TimeSeriesPreprocessor(
    **column_specifiers,
    context_length=context_length,
    prediction_length=forecast_length,
    scaling=True,
    encode_categorical=False,
    scaler_type="standard",
)

train_dataset, valid_dataset, test_dataset = get_datasets(
    tsp,
    data,
    split_params,
)

In [10]:
TTM_MODEL_PATH = "ibm-granite/granite-timeseries-ttm-r1"

zeroshot_model = get_model(
    TTM_MODEL_PATH,
    context_length=context_length,
    prediction_length=forecast_length,
    prediction_channel_indices=tsp.prediction_channel_indices,
    num_input_channels=tsp.num_input_channels,
)

INFO:p-1242115:t-140147142455616:get_model.py:get_model:Loading model from: ibm-granite/granite-timeseries-ttm-r1
INFO:p-1242115:t-140147142455616:get_model.py:get_model:Selected prediction_length = 96
INFO:p-1242115:t-140147142455616:get_model.py:get_model:Model loaded successfully!
INFO:p-1242115:t-140147142455616:get_model.py:get_model:[TTM] context_len = 512, forecast_len = 96


In [11]:
temp_dir = tempfile.mkdtemp()
# zeroshot_trainer
zeroshot_trainer = Trainer(
    model=zeroshot_model,
    args=TrainingArguments(
        output_dir=temp_dir,
        per_device_eval_batch_size=64,
    ),
)

In [12]:
zeroshot_trainer.evaluate(test_dataset)

KeyboardInterrupt: 